Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [42]:
from random import random
from functools import reduce
from collections import namedtuple
from queue import PriorityQueue, SimpleQueue, LifoQueue

import numpy as np

In [43]:
PROBLEM_SIZE = 100
NUM_SETS = 500
SETS = tuple(
    np.array([random() < 1/PROBLEM_SIZE for _ in range(PROBLEM_SIZE)])
    for _ in range(NUM_SETS)
)
State = namedtuple('State', ['taken'])

In [44]:

def get_unique_sets():
    item_covered_by_sets = []
    for i in range(PROBLEM_SIZE):
        item_covered_by_sets.append([])
        for j in range(NUM_SETS):
            if SETS[j][i] == True:
                item_covered_by_sets[i].append(j)
    
    unique_sets = []
    for i, sets_covering in enumerate(item_covered_by_sets):
        if len(sets_covering) == 1:
            unique_sets.append(sets_covering[0])
    print(len(unique_sets))
    return unique_sets
us = get_unique_sets()


7


In [48]:
def goal_check(state):
    return np.all(reduce(
        np.logical_or,
        [SETS[i] for i in state.taken],
        np.array([False for _ in range(PROBLEM_SIZE)]),
    ))

def distance(state):
    # What about “special sets”? -> special set = the only one that has that specific element or that has only one element
    # What about the order of the sets? -> maybe I want the most covering sets first

    uncovered = PROBLEM_SIZE-sum(
        reduce(
            np.logical_or,
            [SETS[i] for i in state.taken],
            np.array([False for _ in range(PROBLEM_SIZE)]),
        ))
    
    unique_sets_covered = 0
    for unique_set in us:
        if unique_set in state.taken:
            unique_sets_covered += 1
        
    return uncovered# - PROBLEM_SIZE*unique_sets_covered

def cost(state):
    return len(state[0])

def astar_cost(state):
    return cost(state)+distance(state)


In [49]:
assert goal_check(
    State(set(range(NUM_SETS)))
), "Probelm not solvable"


In [50]:
frontier = PriorityQueue()
#frontier = SimpleQueue()
state = State(set())
frontier.put((astar_cost(state), state))

counter = 0
_, current_state = frontier.get()
while not goal_check(current_state):
    counter += 1
    for action in range(NUM_SETS):
        if action in current_state.taken:
            continue
        new_state = State(
            current_state.taken ^ {action}
        )
        frontier.put((astar_cost(new_state), new_state))
    _, current_state = frontier.get()
#
print(
    f"Solved in {counter:,} steps ({len(current_state.taken)} tiles)"
)

In [ ]:
current_state

State(taken={5, 389, 7, 136, 272, 145, 274, 20, 149, 23, 283, 40, 42, 47, 53, 58, 192, 66, 67, 69, 74, 342, 87, 216, 488, 115, 253})